In [11]:
from lusidtools.jupyter_tools import toggle_code

"""Entitlements based on access metadata

Demonstrates the use of access metadata to grant access to portfolios in LUSID.

Attributes
----------
entitlements
portfolios
access metadata
"""

toggle_code("Toggle Docstring")

In [12]:
# Import Libraries
import os
import json
import pytz
import pandas as pd
from datetime import datetime, timedelta
from IPython.core.display import HTML

import lusid
from lusid import models as lm
import finbourne_access
from finbourne_access import models as access_models
import finbourne_identity
from finbourne_identity import models as identity_models
from lusidjam import RefreshingToken

# Set DataFrame display formats
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.2f}".format
display(HTML("<style>.container { width:90% !important; }</style>"))

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

lusid_api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

api_client = lusid_api_factory.api_client

lusid_api_url = api_client.configuration.host
access_api_url = lusid_api_url[: lusid_api_url.rfind("/") + 1] + "access"
identity_api_url = lusid_api_url[: lusid_api_url.rfind("/") + 1] + "identity"

access_api_factory = finbourne_access.utilities.ApiClientFactory(
    token=api_client.configuration.access_token,
    access_url=access_api_url,
    app_name="LusidJupyterNotebook",
)

identity_api_factory = finbourne_identity.utilities.ApiClientFactory(
    token=api_client.configuration.access_token,
    api_url=identity_api_url,
    app_name="LusidJupyterNotebook",
)

api_status = pd.DataFrame(
    lusid_api_factory.build(lusid.api.ApplicationMetadataApi)
    .get_lusid_versions()
    .to_dict()
)

display(api_status)

,api_version,build_version,excel_version,links
0,v0,0.6.7304.0,0.5.2237,"{'relation': 'RequestLogs', 'href': 'http://de..."


In [13]:
# Initialise all APIs used in the notebook
data_types_api = lusid_api_factory.build(lusid.DataTypesApi)
properties_api = lusid_api_factory.build(lusid.PropertyDefinitionsApi)
transaction_portfolios_api = lusid_api_factory.build(lusid.TransactionPortfoliosApi)
portfolios_api = lusid_api_factory.build(lusid.PortfoliosApi)
policies_api = access_api_factory.build(finbourne_access.PoliciesApi)
access_roles_api = access_api_factory.build(finbourne_access.RolesApi)
identity_roles_api = identity_api_factory.build(finbourne_identity.RolesApi)
legal_entities_api = lusid_api_factory.build(lusid.LegalEntitiesApi)
propert_definitions_api=lusid_api_factory.build(lusid.PropertyDefinitionsApi)

# (1) Create custom property definition

In [21]:
Prop_request=lm.CreatePropertyDefinitionRequest(scope='LE_Example', 
                                                code='ExternalId', 
                                                value_required=True, 
                                                display_name='Access Property', 
                                                life_time='Perpetual', 
                                                property_description='Access Property',
                                                domain='LegalEntity',
                                                data_type_id=lm.ResourceId(
                code='string',
                scope='system'
            ), constraint_style='Identifier')
try:
   propert_definitions_api.create_property_definition(Prop_request)
except finbourne_identity.ApiException as e:
    detail = json.loads(e.body)
    if detail["code"] != 124: # Role with code already exists
        raise e

ApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Wed, 30 Jun 2021 12:17:50 GMT', 'Content-Type': 'application/problem+json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'X-Rate-Limit-Limit': '1m', 'X-Rate-Limit-Remaining': '4994', 'X-Rate-Limit-Reset': '2021-06-30T12:18:09.0576754Z', 'lusid-meta-success': 'False', 'lusid-meta-requestId': '0HM9RNDFBOEM1:00000001', 'lusid-meta-correlationId': '0HM9RNDFBOEM1:00000001', 'lusid-meta-duration': '215', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains', 'Server': 'FINBOURNE', 'Content-Security-Policy': "default-src 'self' https://*.lusid.com https://*.finbourne.com; script-src 'unsafe-inline' 'self' https://*.lusid.com https://*.finbourne.com; font-src 'self' fonts.googleapis.com; img-src data: 'self' https://*.lusid.com https://*.finbourne.com; style-src 'unsafe-inline' 'self' https://*.lusid.com https://*.finbourne.com; report-uri https://lusid.report-uri.com/r/d/csp/enf

## (2) Create legal entity - With Metadata

In [15]:
identifiers= {
    "legalEntity/LE_Example/ExternalId": {
      "key": "LegalEntity/LE_Example/ExternalId",
      "value": {
        "labelValue": "12345"
      }
    }
  }
            

display_name= "Example Entity" 
description= "Example Entity Description"

legal_entity_request= lm.UpsertLegalEntityRequest(identifiers=identifiers, 
                                                  display_name=display_name,description=description)

legal_entities_api.upsert_legal_entity(legal_entity_request)

{'description': 'Example Entity Description',
 'display_name': 'Example Entity',
 'href': 'https://demosetup.lusid.com/api/api/legalentities/LE_Example/ExternalId/12345/?effectiveAt=2021-06-30T12%3A17%3A24.6493670%2B00%3A00&asAt=2021-06-30T12%3A17%3A24.8515750%2B00%3A00',
 'identifiers': {'LegalEntity/LE_Example/ExternalId': {'effective_from': datetime.datetime(1, 1, 1, 0, 0, tzinfo=tzlocal()),
                                                       'effective_until': datetime.datetime(9999, 12, 31, 23, 59, 59, 999999, tzinfo=tzlocal()),
                                                       'key': 'LegalEntity/LE_Example/ExternalId',
                                                       'value': {'label_value': '12345',
                                                                 'label_value_set': None,
                                                                 'metric_value': None}}},
 'links': [{'description': None,
            'href': 'https://demosetup.lusid.com/api/api

# 2.1 Upsert Metadata to Entity

In [16]:
metadata=lm.AccessMetadataValue(value='PermissionA' , provider='Lusid')
Access_Metadata_Request=lm.UpsertLegalEntityAccessMetadataRequest(metadata=[metadata])
legal_entities_api.upsert_legal_entity_access_metadata('LE_Example', 'ExternalId', "12345", 'Legal_Entity_Metadata', Access_Metadata_Request)

{'href': 'https://demosetup.lusid.com/api/api/persons/LE_Example/ExternalId/12345/metadata/Legal_Entity_Metadata',
 'links': [{'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://demosetup.lusid.com/app/insights/logs/0HM9RNDKKGH2J:00000001',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'next_page': None,
 'previous_page': None,
 'values': [[{'provider': 'Lusid', 'value': 'PermissionA'}]]}

# 2.2 Create Entity without metadata

In [17]:
identifiers= {
    "legalEntity/LE_Example/ExternalId": {
      "key": "LegalEntity/LE_Example/ExternalId",
      "value": {
        "labelValue": "54321"
      }
    }
  }
            

display_name= "Example Entity without metadata" 
description= "Entity without metadata"

legal_entity_request= lm.UpsertLegalEntityRequest(identifiers=identifiers, 
                                                  display_name=display_name,description=description)

legal_entities_api.upsert_legal_entity(legal_entity_request)

{'description': 'Entity without metadata',
 'display_name': 'Example Entity without metadata',
 'href': 'https://demosetup.lusid.com/api/api/legalentities/LE_Example/ExternalId/54321/?effectiveAt=2021-06-30T12%3A17%3A26.1987660%2B00%3A00&asAt=2021-06-30T12%3A17%3A26.5631630%2B00%3A00',
 'identifiers': {'LegalEntity/LE_Example/ExternalId': {'effective_from': datetime.datetime(1, 1, 1, 0, 0, tzinfo=tzlocal()),
                                                       'effective_until': datetime.datetime(9999, 12, 31, 23, 59, 59, 999999, tzinfo=tzlocal()),
                                                       'key': 'LegalEntity/LE_Example/ExternalId',
                                                       'value': {'label_value': '54321',
                                                                 'label_value_set': None,
                                                                 'metric_value': None}}},
 'links': [{'description': None,
            'href': 'https://demosetup.lus

# (3) Create a Policy for access to legal entity metadata

In [48]:
# WhenSpec objects specify the "lifetime" of a modification:
# when it is activated and when it is deactivated.
when_spec = access_models.WhenSpec(
    activate=datetime.now(tz=pytz.utc) - timedelta(days=2),
    deactivate=datetime(9999, 12, 31, tzinfo=pytz.utc),
)

In [49]:
# Create the metadata policy using the policies api
try:
    policies_api.create_policy(
        access_models.PolicyCreationRequest(
            code='Legal_Entity_PermissionA_Access',
            applications=["LUSID"],
            grant=access_models.Grant.ALLOW,
            selectors=[access_models.SelectorDefinition(
                metadata_selector_definition=access_models.MetadataSelectorDefinition(
                    expressions=[
                        access_models.MetadataExpression(
                            metadata_key='Legal_Entity_Metadata',
                            operator=access_models.Operator.EQUALS,
                            text_value="PermissionA"
                        )
                    ],
                    actions=[
                        access_models.ActionId(
                            scope="default",
                            activity="Read",
                            entity="LegalEntity"
                        ),
                        access_models.ActionId(
                            scope="default",
                            activity="List",
                            entity="LegalEntity"
                        )
                    ]
                )
            )],
            when=when_spec
        )
    )
    
except finbourne_access.ApiException as e:
    detail = json.loads(e.body)
    if detail["code"] != 613: # PolicyWithCodeAlreadyExists
        raise e

# (4) Add policies to role

In [55]:
# Create the role using the access API
try:
    access_roles_api.create_role(
        role_creation_request=access_models.RoleCreationRequest(
            code='Legal_Entity_PermissionA_Access_role',
            description='Legal_Entity_PermissionA_Access',
            resource=access_models.RoleResourceRequest(
                policy_id_role_resource=access_models.PolicyIdRoleResource(
                    # Here we apply the policy we defined earlier as well as a default policy to provide basic access
                    policies=[
                        access_models.PolicyId(
                            scope="default",
                            code='Legal_Entity_PermissionA_Access'),
                        access_models.PolicyId(
                            scope="default",
                            code="allow-standard-lusid-features-access"),
                        access_models.PolicyId(
                            scope="default",
                            code="legal_entity")
                    ]
                )
            ),
            when=when_spec
        )
    )

except finbourne_access.ApiException as e:
    detail = json.loads(e.body)
    if detail["code"] != 613: # Role with code already exists
        raise e


# Create the same role using the identity API
try:
    identity_roles_api.create_role(
        create_role_request=identity_models.CreateRoleRequest(
            name='Legal_Entity_PermissionA_Access_role'
        )
    )

except finbourne_identity.ApiException as e:
    detail = json.loads(e.body)
    if detail["code"] != 157: # Role with code already exists
        raise e

# (5) Assign role to user in UI